In [9]:
%pip install pyvis
%pip install pyppeteer

StatementMeta(, 5eaf4197-12de-4933-ad40-7944a0b939b2, 22, Finished, Available, Finished)


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [10]:
import requests
import datetime
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, BooleanType
import sempy.fabric as fabric
import pandas as pd
from pyspark.sql.functions import col
#from pyspark.sql.types import StructType, StructField, StringType

# For visualization:
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network  # pip install pyvis
from IPython.display import HTML, display
import base64

# Instantiate the Fabric REST client (if needed)
FabricAPIClient = fabric.FabricRestClient()

# Initialize Spark session (Fabric notebooks already have Spark available)
spark = SparkSession.builder.getOrCreate()

# ----------------------------------------------------------------------
# Helper Function: Paginated API Call with Token-Based Logic
# ----------------------------------------------------------------------
def get_api_data_paginated(url, params=None, headers=None, apicallforwsmapping=None):
    try:
        token = url.rstrip('/').split('/')[-1].lower()
    except Exception as e:
        print(f"Error extracting token from URL '{url}': {e}")
        token = None
    print(f"API Name: {token}")
    
    all_records = []
    while url:
        if params:
            query_string = "&".join([f"{key}={value}" for key, value in params.items()])
            full_url = f"{url}?{query_string}"
        else:
            full_url = url
        response = requests.get(full_url, headers=headers)
        if response.status_code != 200:
            print(f"Error calling {full_url}: {response.status_code} {response.text}")
            break
        data = response.json()
        if token in ["domains", "workspaces"]:
            if apicallforwsmapping:
                records = data.get("value", [])
            else:
                records = data.get(token, [])
        elif token in ["activityevents"]:
            records = data.get("activityEventEntities", [])
        else:
            records = data.get("value", [])
        all_records.extend(records)
        url = data.get("continuationUri")
        params = None
    return {"value": all_records}

# ----------------------------------------------------------------------
# Helper Function: Split Time Range by UTC Day (Same-Day Chunks)
# ----------------------------------------------------------------------
def split_time_range_by_utc_day(start_dt, end_dt):
    chunks = []
    current = start_dt.replace(microsecond=0)
    while current <= end_dt:
        day_end = datetime.datetime.combine(current.date(), datetime.time(23, 59, 59))
        chunk_start = current
        chunk_end = min(day_end, end_dt)
        chunks.append((chunk_start.isoformat() + "Z", chunk_end.isoformat() + "Z"))
        current = datetime.datetime.combine(current.date() + datetime.timedelta(days=1), datetime.time(0, 0, 0))
    return chunks

# ----------------------------------------------------------------------
# Helper: Safe Create DataFrame from API Response Using an Explicit Schema
# ----------------------------------------------------------------------
def safe_create_df(api_response, default_schema):
    data = api_response.get("value", [])
    return spark.createDataFrame(data, schema=default_schema)

# -------------------------------
# Define Schemas
# -------------------------------
tenant_settings_schema = StructType([
    StructField("tenantSettingGroup", StringType(), True),
    StructField("settingName", StringType(), True),
    StructField("Description", StringType(), True),
    StructField("title", StringType(), True),
    StructField("enabled", BooleanType(), True),
    StructField("canSpecifySecurityGroups", BooleanType(), True),
    StructField("enabledSecurityGroups", StringType(), True),
    StructField("excludedSecurityGroups", StringType(), True),
    StructField("delegateToWorkspace", BooleanType(), True),
    StructField("delegateToCapacity", BooleanType(), True),
    StructField("delegateToDomain", BooleanType(), True),
    StructField("properties", StringType(), True)
])

capacities_schema = StructType([
    StructField("id", StringType(), True),
    StructField("displayName", StringType(), True),
    StructField("sku", StringType(), True),
    StructField("region", StringType(), True),
    StructField("state", StringType(), True)
])

domains_schema = StructType([
    StructField("id", StringType(), True),
    StructField("displayName", StringType(), True),
    StructField("description", StringType(), True),
    StructField("parentDomainId", StringType(), True),
    StructField("contributorsScope", StringType(), True)
])

workspaces_schema = StructType([
    StructField("id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("state", StringType(), True),
    StructField("type", StringType(), True),
    StructField("capacityId", StringType(), True)
    # Domain info is not included in this API.
])

domain_workspaces_schema = StructType([
    StructField("id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("domainId", StringType(), True)
])

activity_events_schema = StructType([
    StructField("Id", StringType(), True),
    StructField("CreationTime", StringType(), True),
    StructField("Operation", StringType(), True),
    StructField("Activity", StringType(), True),
    StructField("UserId", StringType(), True),
    StructField("ClientIP", StringType(), True),
    StructField("ObjectId", StringType(), True),
    StructField("DomainId", StringType(), True),
    StructField("CapacityId", StringType(), True),
    StructField("WorkspaceId", StringType(), True),
    StructField("TenantAdminFeatureSwitches", StringType(), True),
    StructField("Switch", StringType(), True)
])

activity_nodes_schema = StructType([
    StructField("node_id", StringType(), True),
    StructField("node_type", StringType(), True)
])

# ----------------------------------------------------------------------
# New Helper: Get Domain Workspace Mapping as a DataFrame
# ----------------------------------------------------------------------
def get_domain_workspace_mapping(headers):
    mapping = []
    domains_resp = get_api_data_paginated("https://api.fabric.microsoft.com/v1/admin/domains", headers=headers)
    domains = domains_resp.get("value", [])
    for domain in domains:
        domain_id = domain.get("id")
        if not domain_id:
            continue
        url = f"https://api.fabric.microsoft.com/v1/admin/domains/{domain_id}/workspaces"
        resp = get_api_data_paginated(url, headers=headers, apicallforwsmapping=True)
        workspaces = resp.get("value", [])
        for w in workspaces:
            ws_id = w.get("id")
            if ws_id:
                mapping.append({"id": ws_id, "domainId": domain_id})
    return spark.createDataFrame(mapping, schema=StructType([
        StructField("id", StringType(), True),
        StructField("domainId", StringType(), True)
    ]))

# ----------------------------------------------------------------------
# Main Function: Get Activity Events in Chunks with an Inner Loop for Activity Types
# ----------------------------------------------------------------------
def get_activity_events_in_chunks(last_hours=24):
    end_dt = datetime.datetime.utcnow().replace(microsecond=0)
    start_dt = end_dt - datetime.timedelta(hours=last_hours)
    chunks = split_time_range_by_utc_day(start_dt, end_dt)
    print("Time range chunks (start, end):")
    for c in chunks:
        print(c)
    
    all_events = []
    activity_events_url = "https://api.powerbi.com/v1.0/myorg/admin/activityevents"
    activity_types = [
        "UpdateCapacityTenantSettingDelegation",
        "UpdateDomainTenantSettingDelegation",
        "UpdateWorkspaceTenantSettingDelegation",
        "UpdatedAdminFeatureSwitch"
    ]
    
    url_for_token = "https://analysis.windows.net/powerbi/api"
    token = mssparkutils.credentials.getToken(url_for_token)
    headers_local = {
        "Authorization": "Bearer " + token,
        "Content-Type": "application/json"
    }
    
    for (chunk_start, chunk_end) in chunks:
        for activity in activity_types:
            activity_params = {
                "startDateTime": f"'{chunk_start}'",
                "endDateTime": f"'{chunk_end}'",
                "$filter": f"Activity eq '{activity}'"
            }
            response = get_api_data_paginated(activity_events_url, params=activity_params, headers=headers_local)
            events = response.get("value", [])
            all_events.extend(events)
    
    return all_events

# ----------------------------------------------------------------------
# Build Minimal Graph Tables and Detailed Dimension Tables
# ----------------------------------------------------------------------
tenant_node = spark.createDataFrame([("tenant", "Tenant")], ["node_id", "node_type"])

tenant_settings_url = "https://api.fabric.microsoft.com/v1/admin/tenantsettings"
capacity_overrides_url = "https://api.fabric.microsoft.com/v1/admin/capacities/delegatedTenantSettingOverrides"
list_capacities_url = "https://api.fabric.microsoft.com/v1/capacities"
domain_overrides_url = "https://api.fabric.microsoft.com/v1/admin/domains/delegatedTenantSettingOverrides"
workspace_overrides_url = "https://api.fabric.microsoft.com/v1/admin/workspaces/delegatedTenantSettingOverrides"
list_domains_url = "https://api.fabric.microsoft.com/v1/admin/domains"
list_workspaces_url = "https://api.fabric.microsoft.com/v1/admin/workspaces"

url_for_token = "https://analysis.windows.net/powerbi/api"
token = mssparkutils.credentials.getToken(url_for_token)
headers = {
    "Authorization": "Bearer " + token,
    "Content-Type": "application/json"
}

# Tenant Settings.
tenant_settings_resp = get_api_data_paginated(tenant_settings_url, headers=headers)
tenant_settings_df = safe_create_df(tenant_settings_resp, tenant_settings_schema) \
    .withColumn("node_id", F.concat(F.lit("tenant_setting_"), F.col("settingName"))) \
    .withColumn("node_type", F.lit("tenant_setting"))

# List of Capacities. Normalize IDs to lower-case.
capacities_resp = get_api_data_paginated(list_capacities_url, headers=headers)
capacities_df = safe_create_df(capacities_resp, capacities_schema) \
    .withColumn("node_id", F.concat(F.lit("capacity_"), F.lower(F.col("id")))) \
    .withColumn("node_type", F.lit("capacity"))

# List of Domains.
domains_resp = get_api_data_paginated(list_domains_url, headers=headers)
domains_df = safe_create_df(domains_resp, domains_schema) \
    .withColumn("node_id", F.concat(F.lit("domain_"), F.col("id"))) \
    .withColumn("node_type", F.lit("domain"))

# List of Workspaces.
workspaces_resp = get_api_data_paginated(list_workspaces_url, headers=headers)
workspaces_df = safe_create_df(workspaces_resp, workspaces_schema) \
    .withColumn("node_id", F.concat(F.lit("workspace_"), F.col("id"))) \
    .withColumn("node_type", F.lit("workspace"))

# Update workspaces_df with Domain Membership.
mapping_df = get_domain_workspace_mapping(headers)
workspaces_df = workspaces_df.join(mapping_df, on="id", how="left")

# List of Capacity Overrides.
capacity_overrides_resp = get_api_data_paginated(capacity_overrides_url, headers=headers)
flattened_records = []
for rec in capacity_overrides_resp.get("value", []):
    capacity_id = rec.get("id")
    tenant_settings_list = rec.get("tenantSettings", [])
    for ts in tenant_settings_list:
        flattened_records.append({
            "capacityId": capacity_id,
            "settingName": ts.get("settingName")
        })
cap_schema = StructType([
    StructField("capacityId", StringType(), True),
    StructField("settingName", StringType(), True)
])
capacity_overrides_df = spark.createDataFrame(flattened_records, schema=cap_schema)
capacity_overrides_df.show(truncate=False)

# List of Domain Overrides.
domain_overrides_resp = get_api_data_paginated(domain_overrides_url, headers=headers)
flattened_domain_records = []
for rec in domain_overrides_resp.get("value", []):
    domain_id = rec.get("id")
    tenant_Do_settings = rec.get("tenantSettings", [])
    for ts in tenant_Do_settings:
        flattened_domain_records.append({
            "domainId": domain_id,
            "settingName": ts.get("settingName")
        })
dom_schema = StructType([
    StructField("domainId", StringType(), True),
    StructField("settingName", StringType(), True)
])
domain_overrides_df = spark.createDataFrame(flattened_domain_records, schema=dom_schema)
domain_overrides_df.show(truncate=False)

# List of Workspace Overrides.
workspace_overrides_resp = get_api_data_paginated(workspace_overrides_url, headers=headers)
flattened_workspace_records = []
for rec in workspace_overrides_resp.get("value", []):
    workspace_id = rec.get("id")
    tenant_Wo_settings = rec.get("tenantSettings", [])
    for ts in tenant_Wo_settings:
        flattened_workspace_records.append({
            "workspaceId": workspace_id,
            "settingName": ts.get("settingName")
        })
ws_schema = StructType([
    StructField("workspaceId", StringType(), True),
    StructField("settingName", StringType(), True)
])
workspace_overrides_df = spark.createDataFrame(flattened_workspace_records, schema=ws_schema)
workspace_overrides_df.show(truncate=False)

# Activity Events.
try:
    activity_events = get_activity_events_in_chunks(last_hours=150)
    activity_events_array = []
    if activity_events:
        for rec in activity_events:
            record = {}
            record["Id"] = rec.get("Id")
            record["CreationTime"] = rec.get("CreationTime")
            record["Operation"] = rec.get("Operation")
            record["Activity"] = rec.get("Activity")
            record["UserId"] = rec.get("UserId")
            record["ClientIP"] = rec.get("ClientIP")
            record["ObjectId"] = rec.get("ObjectId")
            record["DomainId"] = rec.get("DomainId")
            record["CapacityId"] = rec.get("CapacityId")
            record["WorkspaceId"] = rec.get("WorkspaceId")
            switches = rec.get("TenantAdminFeatureSwitches", [])
            if switches and isinstance(switches, list) and len(switches) > 0:
                record["TenantAdminFeatureSwitches"] = switches[0]
                record["Switch"] = switches[0].get("SwitchName")
                record["ObjectId"] = record["Switch"]  # For visual purposes.
            else:
                record["TenantAdminFeatureSwitches"] = None
                record["Switch"] = None
            activity_events_array.append(record)
        activity_df = spark.createDataFrame(activity_events_array, schema=activity_events_schema) \
            .withColumn("node_id", F.concat(F.lit("activity_"), F.col("Id"))) \
            .withColumn("node_type", F.lit("activity"))
    else:
        empty_activity_schema = StructType([
            StructField("Id", StringType(), True),
            StructField("CreationTime", StringType(), True),
            StructField("Operation", StringType(), True),
            StructField("Activity", StringType(), True),
            StructField("UserId", StringType(), True),
            StructField("ClientIP", StringType(), True),
            StructField("ObjectId", StringType(), True),
            StructField("DomainId", StringType(), True),
            StructField("CapacityId", StringType(), True),
            StructField("WorkspaceId", StringType(), True),
            StructField("TenantAdminFeatureSwitches", StringType(), True),
            StructField("Switch", StringType(), True)
        ])
        activity_df = spark.createDataFrame([], schema=empty_activity_schema)
except Exception as e:
    print(f"Error extracting activity events: {e}")
    empty_activity_schema = StructType([
        StructField("Id", StringType(), True),
        StructField("CreationTime", StringType(), True),
        StructField("Operation", StringType(), True),
        StructField("Activity", StringType(), True),
        StructField("UserId", StringType(), True),
        StructField("ClientIP", StringType(), True),
        StructField("ObjectId", StringType(), True),
        StructField("DomainId", StringType(), True),
        StructField("CapacityId", StringType(), True),
        StructField("WorkspaceId", StringType(), True),
        StructField("TenantAdminFeatureSwitches", StringType(), True),
        StructField("Switch", StringType(), True)
    ])
    activity_df = spark.createDataFrame([], schema=empty_activity_schema)

print("Finished extracting activity events")

# ----------------------------------------------------------------------
# Build Graph Edges
# ----------------------------------------------------------------------
edges_tenant_settings = tenant_settings_df.select(F.lit("tenant").alias("src_node"),
                                                  F.col("node_id").alias("dest_node")) \
                                           .withColumn("relationship", F.lit("defines"))

edges_tenant_domains = domains_df.select(F.lit("tenant").alias("src_node"),
                                         F.col("node_id").alias("dest_node")) \
                                  .withColumn("relationship", F.lit("contains_domain"))

edges_tenant_capacities = capacities_df.select(F.lit("tenant").alias("src_node"),
                                               F.col("node_id").alias("dest_node")) \
                                        .withColumn("relationship", F.lit("contains_capacity"))

edges_tenant_direct_workspaces = workspaces_df.filter((F.col("capacityId").isNull()) | (F.col("capacityId") == "")) \
    .filter(F.col("domainId").isNull() | (F.col("domainId") == "")) \
    .select(F.lit("tenant").alias("src_node"),
            F.col("node_id").alias("dest_node")) \
    .withColumn("relationship", F.lit("contains_workspace_direct"))

'''
if not activity_df.rdd.isEmpty():
    edges_settings_activity = activity_df.alias("act") \
        .join(tenant_settings_df.alias("ts"), F.col("act.Activity") == F.col("ts.settingName"), "inner") \
        .select(F.concat(F.lit("tenant_setting_"), F.col("ts.settingName")).alias("src_node"),
                F.col("act.node_id").alias("dest_node")) \
        .withColumn("relationship", F.lit("activity_modification"))
else:
    from pyspark.sql.types import StructType, StructField, StringType
    empty_edges_schema = StructType([
        StructField("src_node", StringType(), True),
        StructField("dest_node", StringType(), True),
        StructField("relationship", StringType(), True)
    ])
    edges_settings_activity = spark.createDataFrame([], schema=empty_edges_schema)

edges_settings_activity.show()'''

edges_domains_workspaces = workspaces_df.filter(F.col("domainId").isNotNull()) \
    .select(F.concat(F.lit("domain_"), F.col("domainId")).alias("src_node"),
            F.col("node_id").alias("dest_node")) \
    .withColumn("relationship", F.lit("contains_workspace"))
    
edges_overridden_by_domain = domain_overrides_df.withColumn("src_node", F.concat(F.lit("domain_"), F.col("domainId"))) \
    .withColumn("dest_node", F.concat(F.lit("tenant_setting_"), F.col("settingName"), F.lit("_domain_"), F.col("domainId"))) \
    .withColumn("relationship", F.lit("overridden_by_domain")) \
    .select("src_node", "dest_node", "relationship")
    
edges_capacities_workspaces = workspaces_df.filter(F.col("capacityId").isNotNull() & (F.col("capacityId") != "")) \
    .select(F.concat(F.lit("capacity_"), F.lower(F.col("capacityId"))).alias("src_node"),
            F.col("node_id").alias("dest_node")) \
    .withColumn("relationship", F.lit("contains_workspace"))
    
edges_overridden_by_capacity = capacity_overrides_df.withColumn("src_node", F.concat(F.lit("capacity_"), F.lower(F.col("capacityId")))) \
    .withColumn("dest_node", F.concat(F.lit("tenant_setting_"), F.col("settingName"), F.lit("_capacity_"), F.lower(F.col("capacityId")))) \
    .withColumn("relationship", F.lit("overridden_by_capacity")) \
    .select("src_node", "dest_node", "relationship")
    
edges_overridden_by_workspace = workspace_overrides_df.withColumn("src_node", F.concat(F.lit("workspace_"), F.col("workspaceId"))) \
    .withColumn("dest_node", F.concat(F.lit("tenant_setting_"), F.col("settingName"), F.lit("_workspace_"), F.col("workspaceId"))) \
    .withColumn("relationship", F.lit("overridden_by_workspace")) \
    .select("src_node", "dest_node", "relationship")
    
'''edges_workspaces_overrides = workspace_overrides_df.withColumn("src_node", F.concat(F.lit("workspace_"), F.col("workspaceId"))) \
    .withColumn("dest_node", F.concat(F.lit("tenant_setting_"), F.col("settingName"), F.lit("_workspace_"), F.col("workspaceId"))) \
    .withColumn("relationship", F.lit("overridden_by_workspace_deep")) \
    .select("src_node", "dest_node", "relationship")'''

# --- Create Activity-to-TenantSetting Edges Based on Operation ---

# 1. Tenant-level activity edges:
edges_tenant_activity= (
    activity_df.filter(F.col("Operation") == "UpdatedAdminFeatureSwitch")
    .alias("act")
    .withColumn("src_node", F.concat(F.lit("tenant_setting_"), F.col("act.ObjectId"))) \
    .withColumn("dest_node", F.concat(F.lit("activity_"),F.col("act.Id"))) \
    .withColumn("relationship", F.lit("modified_by")) \
    .select("src_node", "dest_node", "relationship")
    
)
edges_domain_activity= (
    activity_df.filter(F.col("Operation") == "UpdateDomainTenantSettingDelegation")
    .alias("act")
    .withColumn("src_node", F.concat(F.lit("tenant_setting_"), F.col("act.ObjectId"), F.lit("_domain_"), F.col("act.DomainId"))) \
    .withColumn("dest_node", F.concat(F.lit("activity_"),F.col("act.Id"))) \
    .withColumn("relationship", F.lit("modified_by")) \
    .select("src_node", "dest_node", "relationship")    
)
edges_capacity_activity= (
    activity_df.filter(F.col("Operation") == "UpdateCapacityTenantSettingDelegation")
    .alias("act")
    .withColumn("src_node", F.concat(F.lit("tenant_setting_"), F.col("act.ObjectId"), F.lit("_capacity_"), F.col("act.CapacityId"))) \
    .withColumn("dest_node", F.concat(F.lit("activity_"),F.col("act.Id"))) \
    .withColumn("relationship", F.lit("modified_by")) \
    .select("src_node", "dest_node", "relationship")    
)
edges_workspace_activity= (
    activity_df.filter(F.col("Operation") == "UpdateWorkspaceTenantSettingDelegation")
    .alias("act")
    .withColumn("src_node", F.concat(F.lit("tenant_setting_"), F.col("act.ObjectId"), F.lit("_workspace_"), F.col("act.WorkspaceId"))) \
    .withColumn("dest_node", F.concat(F.lit("activity_"),F.col("act.Id"))) \
    .withColumn("relationship", F.lit("modified_by")) \
    .select("src_node", "dest_node", "relationship")    
)
'''

        "UpdateCapacityTenantSettingDelegation",
        "UpdateDomainTenantSettingDelegation",
        "UpdateWorkspaceTenantSettingDelegation",
        "UpdatedAdminFeatureSwitch"



'''
print("Finished creating activity edges and extended edge set.")
    
edges_all = edges_tenant_settings.union(edges_tenant_domains) \
    .union(edges_tenant_capacities) \
    .union(edges_tenant_direct_workspaces) \
    .union(edges_domains_workspaces) \
    .union(edges_capacities_workspaces) \
    .union(edges_overridden_by_domain) \
    .union(edges_overridden_by_capacity) \
    .union(edges_overridden_by_workspace) \
    .union(edges_tenant_activity)\
    .union(edges_domain_activity)\
    .union(edges_workspace_activity)\
    .union(edges_capacity_activity)

print("Finished creating edges")

from pyspark.sql.types import StructType, StructField, StringType
if activity_df.rdd.isEmpty():
    activity_nodes = spark.createDataFrame([], schema=activity_nodes_schema)
else:
    activity_nodes = activity_df.select("node_id", "node_type")

nodes_basic = tenant_node.select("node_id", "node_type") \
    .union(tenant_settings_df.select("node_id", "node_type")) \
    .union(capacities_df.select("node_id", "node_type")) \
    .union(domains_df.select("node_id", "node_type")) \
    .union(workspaces_df.select("node_id", "node_type")) \
    .union(activity_nodes)

# Create additional tenant_setting nodes for each override level with unique identifiers.
ts_domain_nodes = domain_overrides_df.select(
    F.concat(F.lit("tenant_setting_"), F.col("settingName"), F.lit("_domain_"), F.col("domainId")).alias("node_id")
).withColumn("node_type", F.lit("tenant_setting"))

ts_capacity_nodes = capacity_overrides_df.select(
    F.concat(F.lit("tenant_setting_"), F.col("settingName"), F.lit("_capacity_"), F.lower(F.col("capacityId"))).alias("node_id")
).withColumn("node_type", F.lit("tenant_setting"))

ts_workspace_nodes = workspace_overrides_df.select(
    F.concat(F.lit("tenant_setting_"), F.col("settingName"), F.lit("_workspace_"), F.col("workspaceId")).alias("node_id")
).withColumn("node_type", F.lit("tenant_setting"))

nodes_all = nodes_basic.union(ts_domain_nodes).union(ts_capacity_nodes).union(ts_workspace_nodes)
nodes_all.show(truncate=False)

print("Finished creating nodes")

dim_tenant_settings = tenant_settings_df
dim_capacities = capacities_df
dim_domains = domains_df
dim_workspaces = workspaces_df
dim_activity_events = activity_df

fact_domains = domain_overrides_df
fact_capacity = capacity_overrides_df
fact_workspace = workspace_overrides_df

print("Finished creating dimension tables")

fact_table_path = "graph_fact_table"
fact_capacity_override_table_path = "fact_capacity_override_table"
fact_domain_override_table_path = "fact_domain_override_table"
fact_workspace_override_table_path = "fact_workspace_override_table"
dim_nodes_path = "dim_nodes"
dim_tenant_settings_path = "dim_tenant_settings"
dim_capacities_path = "dim_capacities"
dim_domains_path = "dim_domains"
dim_workspaces_path = "dim_workspaces"
dim_activity_events_path = "dim_activity_events"

edges_all.write.format("delta").mode("overwrite").saveAsTable(fact_table_path)
nodes_all.write.format("delta").mode("overwrite").saveAsTable(dim_nodes_path)

fact_domains.write.format("delta").mode("overwrite").saveAsTable(fact_domain_override_table_path)
fact_capacity.write.format("delta").mode("overwrite").saveAsTable(fact_capacity_override_table_path)
fact_workspace.write.format("delta").mode("overwrite").saveAsTable(fact_workspace_override_table_path)

dim_tenant_settings.write.format("delta").mode("overwrite").saveAsTable(dim_tenant_settings_path)
dim_capacities.write.format("delta").mode("overwrite").saveAsTable(dim_capacities_path)
dim_domains.write.format("delta").mode("overwrite").saveAsTable(dim_domains_path)
dim_workspaces.write.format("delta").mode("overwrite").saveAsTable(dim_workspaces_path)
dim_activity_events.write.format("delta").mode("overwrite").saveAsTable(dim_activity_events_path)

print("Graph fact table, minimal nodes, and dimension tables have been written to the Lakehouse.")



StatementMeta(, 5eaf4197-12de-4933-ad40-7944a0b939b2, 24, Finished, Available, Finished)

API Name: tenantsettings
API Name: capacities
API Name: domains
API Name: workspaces
API Name: domains
API Name: workspaces
API Name: workspaces
API Name: delegatedtenantsettingoverrides
+----------+-----------+
|capacityId|settingName|
+----------+-----------+
+----------+-----------+

API Name: delegatedtenantsettingoverrides
+------------------------------------+------------------+
|domainId                            |settingName       |
+------------------------------------+------------------+
|26174495-3e51-467b-90bc-7c42413ebe30|ExportToWord      |
|77990749-c7b8-42f5-affb-c88ba6a86c76|ExportToPowerPoint|
|77990749-c7b8-42f5-affb-c88ba6a86c76|ExportToWord      |
|77990749-c7b8-42f5-affb-c88ba6a86c76|ExportToXML       |
|77990749-c7b8-42f5-affb-c88ba6a86c76|ExportToImage     |
+------------------------------------+------------------+

API Name: delegatedtenantsettingoverrides
+------------------------------------+------------------+
|workspaceId                         |settingNa

In [11]:
from IPython.display import display
from pyspark.sql.functions import col
from collections import defaultdict

# ----------------------------------------------------------------------
# Icon Generation Functions
# ----------------------------------------------------------------------
import hashlib
import base64

# ----------------------------------------------------------------------
# Icon Generation Functions
# ----------------------------------------------------------------------
def encode_svg(svg_str):
    b64_svg = base64.b64encode(svg_str.encode('utf-8')).decode('utf-8')
    return f"data:image/svg+xml;base64,{b64_svg}"

# Provided SVG icon strings.
svg_icons = {
"tenant": '''
    <svg fill="#000000" version="1.1" id="Layer_1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" 
        viewBox="0 0 32 32" xml:space="preserve">
        <path id="office_1_" d="M31,31.36H1c-0.199,0-0.36-0.161-0.36-0.36V1c0-0.199,0.161-0.36,0.36-0.36h18c0.199,0,0.36,0.161,0.36,0.36
            v2.64H25c0.199,0,0.36,0.161,0.36,0.36v3.64H31c0.199,0,0.36,0.161,0.36,0.36v23C31.36,31.199,31.199,31.36,31,31.36z M19.36,30.64
            h11.28V8.36H19.36V30.64z M12.36,30.64h6.28V1.36H1.36v29.28h6.28V23c0-0.199,0.161-0.36,0.36-0.36h4c0.199,0,0.36,0.161,0.36,0.36
            V30.64z M8.36,30.64h3.28v-7.28H8.36V30.64z M19.36,7.64h5.279V4.36H19.36V7.64z M27.36,25h-0.72v-2h0.721L27.36,25L27.36,25z
            M23.36,25h-0.72v-2h0.721L23.36,25L23.36,25z M16.36,25h-0.72v-2h0.72V25z M4.36,25H3.64v-2h0.72V25z M27.36,19h-0.72v-2h0.721
            L27.36,19L27.36,19z M23.36,19h-0.72v-2h0.721L23.36,19L23.36,19z M16.36,19h-0.72v-2h0.72V19z M12.36,19h-0.72v-2h0.72V19z
            M8.36,19H7.64v-2h0.72V19z M4.36,19H3.64v-2h0.72V19z M27.36,13h-0.72v-2h0.721L27.36,13L27.36,13z M23.36,13h-0.72v-2h0.721
            L23.36,13L23.36,13z M16.36,13h-0.72v-2h0.72V13z M12.36,13h-0.72v-2h0.72V13z M8.36,13H7.64v-2h0.72V13z M4.36,13H3.64v-2h0.72V13z
            M16.36,7h-0.72V5h0.72V7z M12.36,7h-0.72V5h0.72V7z M8.36,7H7.64V5h0.72V7z M4.36,7H3.64V5h0.72V7z"/>
        <rect id="_Transparent_Rectangle" style="fill:none;" width="32" height="32"/>
    </svg>
''',
"capacity": '''
    <svg width="800px" height="800px" viewBox="0 0 32 32" fill="none" xmlns="http://www.w3.org/2000/svg">
        <g clip-path="url(#clip0_901_2773)">
        <path fill-rule="evenodd" clip-rule="evenodd" d="M31 21V30C31 30.55 30.55 31 30 31H2C1.45 31 1 30.55 1 30V21H31Z" fill="#FFC44D"/>
        <path d="M1 21H31V11H1V21Z" fill="#FFC44D"/>
        <path d="M31 2V11H1V2C1 1.45 1.45 1 2 1H30C30.55 1 31 1.45 31 2Z" fill="#FFC44D"/>
        <path d="M4 21H31V14M7 6H17M7 16H17M7 26H17M4 11H31V2C31 1.447 30.553 1 30 1H2C1.447 1 1 1.447 1 2V30C1 30.553 1.447 31 2 31H30C30.553 31 31 30.553 31 30V24" stroke="#000000" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"/>
        </g>
        <defs>
        <clipPath id="clip0_901_2773">
        <rect width="32" height="32" fill="white"/>
        </clipPath>
        </defs>
    </svg>
''',
    "workspace": '''
    <svg version="1.1" id="Layer_1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" 
	 viewBox="0 0 512.002 512.002" xml:space="preserve">
        <path style="fill:#FEDEA1;" d="M12.8,499.2V64h153.6v435.2H12.8z M89.6,371.2c-14.114,0-25.6,11.486-25.6,25.6
            s11.486,25.6,25.6,25.6s25.6-11.486,25.6-25.6S103.714,371.2,89.6,371.2z"/>
        <rect x="166.4" y="12.8" style="fill:#FFB7B2;" width="153.6" height="102.4"/>
        <path style="fill:#FF877F;" d="M166.4,499.2v-384H320v384H166.4z M243.2,345.6c-14.114,0-25.6,11.486-25.6,25.6
            s11.486,25.6,25.6,25.6c14.114,0,25.6-11.486,25.6-25.6S257.314,345.6,243.2,345.6z"/>
        <g>
            
                <rect x="351.628" y="40.169" transform="matrix(0.9848 -0.1736 0.1736 0.9848 -7.7174 68.9028)" style="fill:#6FB0B6;" width="76.799" height="76.799"/>
            
                <rect x="418.411" y="418.352" transform="matrix(0.9848 -0.1736 0.1736 0.9848 -72.3563 86.239)" style="fill:#6FB0B6;" width="76.799" height="76.79"/>
        </g>
        <circle style="fill:#BDC4D0;" cx="89.6" cy="396.8" r="25.6"/>
        <rect x="385.017" y="114.066" transform="matrix(0.9848 -0.1736 0.1736 0.9848 -40.0378 77.5705)" style="fill:#538489;" width="76.799" height="307.195"/>
        <circle style="fill:#BDC4D0;" cx="243.2" cy="371.2" r="25.6"/>
        <g>
            <path style="fill:#573A32;" d="M89.6,358.4c-21.205,0-38.4,17.195-38.4,38.4c0,21.205,17.195,38.4,38.4,38.4s38.4-17.195,38.4-38.4
                C128,375.595,110.805,358.4,89.6,358.4z M89.6,409.6c-7.057,0-12.8-5.743-12.8-12.8S82.543,384,89.6,384
                c7.057,0,12.8,5.743,12.8,12.8S96.657,409.6,89.6,409.6z"/>
            <path style="fill:#573A32;" d="M243.2,409.6c21.205,0,38.4-17.195,38.4-38.4c0-21.205-17.195-38.4-38.4-38.4
                s-38.4,17.195-38.4,38.4C204.8,392.405,221.995,409.6,243.2,409.6z M243.2,358.4c7.057,0,12.8,5.743,12.8,12.8
                s-5.743,12.8-12.8,12.8s-12.8-5.743-12.8-12.8S236.143,358.4,243.2,358.4z"/>
            <path style="fill:#573A32;" d="M307.2,0h-128c-14.14,0-25.6,11.46-25.6,25.6v25.6h-128C11.46,51.2,0,62.66,0,76.8v409.6
                C0,500.54,11.46,512,25.6,512h128c4.77,0,8.977-1.655,12.8-3.925c3.823,2.27,8.038,3.925,12.8,3.925h128
                c14.14,0,25.6-11.46,25.6-25.6V25.6C332.8,11.46,321.34,0,307.2,0z M25.6,486.4V76.8h128v409.6H25.6z M307.2,486.4h-128V128h128
                V486.4z M307.2,102.4h-128V25.6h128V102.4z"/>
            <path style="fill:#573A32;" d="M436.036,44.467c-2.458-13.918-15.735-23.219-29.662-20.762l-50.415,8.892
                c-13.918,2.458-23.219,15.735-20.762,29.662l75.571,428.587c2.458,13.926,15.735,23.219,29.653,20.762l50.423-8.892
                c13.926-2.458,23.219-15.736,20.762-29.653L436.036,44.467z M360.405,57.805l50.423-8.892l8.892,50.423l-50.423,8.892
                L360.405,57.805z M373.734,133.444l50.423-8.892l48.896,277.325l-50.423,8.892L373.734,133.444z M435.977,486.392l-8.892-50.423
                l50.423-8.892L486.4,477.5L435.977,486.392z"/>
            <rect x="51.2" y="153.6" style="fill:#573A32;" width="76.8" height="25.6"/>
        </g>
    </svg>
''',
    "domain": '''
    <svg height="800px" width="800px" version="1.1" id="Layer_1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" 
	 viewBox="0 0 512.001 512.001" xml:space="preserve">
        <path style="fill:#D35B38;" d="M271.202,42.527c0,0-3.514,6.794,10.542,14.056c5.723,2.957,16.222,5.836,26.659,8.21l1.004-15.395
            c1.384-21.224-15.457-39.199-36.725-39.199h-33.88c-21.268,0-38.109,17.975-36.725,39.199l1.505,23.089
            C256.961,69.518,271.202,42.527,271.202,42.527z"/>
        <path style="fill:#FFE6B8;" d="M281.744,56.582c-14.056-7.262-10.542-14.056-10.542-14.056s-14.241,26.991-67.621,29.96
            l3.516,53.917c1.083,16.609,10.566,31.519,25.151,39.54l0,0c14.628,8.045,32.356,8.045,46.984,0l0,0
            c14.585-8.022,24.068-22.931,25.151-39.54l4.017-61.611C297.965,62.418,287.467,59.539,281.744,56.582z"/>
        <path style="fill:#BCC987;" d="M389.982,263.535c-6.019-33.221-31.949-58.558-64.136-64.619l-70.104,70.104l-70.104-70.104
            c-32.187,6.06-58.118,31.398-64.136,64.619l-5.611,30.972h279.703L389.982,263.535z"/>
        <path style="fill:#FFE6B8;" d="M325.845,198.916c-4.844-0.912-9.827-1.392-14.907-1.392h-22.897l-32.3,32.407l-32.997-32.407h-22.2
            c-5.08,0-10.064,0.479-14.907,1.392l70.104,70.104L325.845,198.916z"/>
        <g>
            <circle style="fill:#D35B38;" cx="57.258" cy="473.814" r="27.985"/>
            <circle style="fill:#D35B38;" cx="255.745" cy="473.814" r="27.985"/>
            <circle style="fill:#D35B38;" cx="454.742" cy="473.814" r="27.985"/>
        </g>
        <g>
            <path style="fill:#4D3D36;" d="M193.404,73.151l3.516,53.916c1.316,20.189,12.686,38.063,30.413,47.815
                c8.894,4.891,18.65,7.336,28.407,7.336s19.514-2.446,28.409-7.337c17.728-9.75,29.097-27.625,30.413-47.814l4.017-61.611
                l1.004-15.393c0.842-12.919-3.75-25.748-12.601-35.196C298.13,5.419,285.628,0,272.68,0H238.8c-12.946,0-25.448,5.419-34.3,14.866
                c-8.853,9.449-13.446,22.276-12.602,35.195L193.404,73.151z M294.207,125.74c-0.861,13.202-8.296,24.89-19.888,31.267
                c-11.635,6.398-25.525,6.397-37.155,0.001c-11.593-6.378-19.029-18.066-19.889-31.268l-2.867-43.969
                c27.76-3.328,44.752-13.139,54.496-21.458c2.266,1.912,4.988,3.691,8.156,5.329c4.5,2.325,11.291,4.648,20.606,7.047
                L294.207,125.74z M299.229,48.735l-0.209,3.202c-7.384-2.056-10.954-3.568-12.595-4.415c-2.781-1.437-4.23-2.605-4.957-3.324
                l6.983-18.624c1.293,0.959,2.515,2.036,3.644,3.24C297.179,34.24,299.713,41.315,299.229,48.735z M219.387,28.813
                c5.083-5.426,11.979-8.414,19.414-8.414h29.806l-6.643,17.715c-1.661,2.546-13.581,18.772-48.886,23.28l-0.826-12.658
                C211.769,41.315,214.303,34.239,219.387,28.813z"/>
            <path style="fill:#4D3D36;" d="M115.889,304.706h279.703c3.023,0,5.89-1.341,7.829-3.661c1.937-2.32,2.746-5.381,2.207-8.356
                l-5.612-30.971c-6.643-36.668-35.691-65.934-72.284-72.823c-5.526-1.041-11.176-1.569-16.795-1.569h-22.897
                c-2.711,0-5.31,1.08-7.224,2.999l-25.152,25.237l-25.774-25.313c-1.907-1.873-4.473-2.922-7.147-2.922h-22.2
                c-5.62,0-11.271,0.528-16.796,1.569c-0.001,0-0.003,0.001-0.004,0.001c-36.59,6.893-65.637,36.156-72.28,72.822l-5.611,30.972
                c-0.539,2.974,0.27,6.035,2.207,8.355C109.999,303.364,112.866,304.706,115.889,304.706z M218.573,207.723l30.022,29.485
                c3.998,3.926,10.414,3.891,14.371-0.076l29.31-29.408h10.339l-46.874,46.873l-46.874-46.874L218.573,207.723L218.573,207.723z
                M131.536,265.354c4.832-26.673,24.967-48.289,50.868-55.246l66.126,66.126c1.992,1.991,4.602,2.987,7.212,2.987
                c2.61,0,5.22-0.996,7.212-2.987l66.125-66.126c25.902,6.957,46.035,28.573,50.866,55.245c0,0,0,0,0,0.001l3.434,18.954H128.103
                L131.536,265.354z"/>
            <path style="fill:#4D3D36;" d="M297.2,349.605h-6.218c-5.632,0-10.199,4.567-10.199,10.199c0,5.632,4.567,10.199,10.199,10.199
                h6.218c5.632,0,10.199-4.567,10.199-10.199C307.399,354.172,302.832,349.605,297.2,349.605z"/>
            <path style="fill:#4D3D36;" d="M265.94,437.027V333.892c0-5.632-4.567-10.199-10.199-10.199s-10.199,4.567-10.199,10.199v15.713
                H93.793c-25.77,0-46.735,20.965-46.735,46.734v40.687c-16.117,4.473-27.985,19.266-27.985,36.79
                C19.073,494.87,36.202,512,57.257,512s38.184-17.13,38.184-38.184c0-17.522-11.868-32.315-27.985-36.79V396.34
                c0-14.522,11.815-26.335,26.336-26.335h151.749v67.023c-16.117,4.473-27.985,19.266-27.985,36.79
                c0,21.055,17.13,38.184,38.184,38.184s38.184-17.13,38.184-38.184C293.925,456.293,282.057,441.5,265.94,437.027z M75.042,473.816
                c0,9.807-7.978,17.785-17.785,17.785s-17.785-7.979-17.785-17.785s7.979-17.785,17.785-17.785S75.042,464.009,75.042,473.816z
                M255.741,491.602c-9.807,0-17.785-7.979-17.785-17.785s7.979-17.785,17.785-17.785c9.807,0,17.785,7.978,17.785,17.785
                S265.547,491.602,255.741,491.602z"/>
            <path style="fill:#4D3D36;" d="M464.425,436.879v-40.54c0-25.769-20.965-46.734-46.735-46.734h-84.213
                c-5.632,0-10.199,4.567-10.199,10.199c0,5.632,4.567,10.199,10.199,10.199h84.213c14.522,0,26.336,11.814,26.336,26.335v40.835
                c-15.853,4.645-27.466,19.307-27.466,36.642c0,21.055,17.129,38.184,38.184,38.184c21.055,0,38.184-17.13,38.184-38.184
                C492.927,456.105,480.806,441.176,464.425,436.879z M454.743,491.602c-9.807,0-17.785-7.979-17.785-17.785
                s7.978-17.785,17.785-17.785c9.808,0,17.785,7.978,17.785,17.785S464.55,491.602,454.743,491.602z"/>
        </g>
    </svg>
''',
    "tenant_settings": '''
    <svg height="800px" width="800px" version="1.1" id="Layer_1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" 
	 viewBox="0 0 512 512" xml:space="preserve">
<path style="fill:#FFE6B8;" d="M463.311,200.043c-4.867-18.074-12.021-35.21-21.155-51.064c14.05-16.402,13.324-41.113-2.204-56.641
	l-20.289-20.289c-15.528-15.528-40.239-16.255-56.641-2.204c-15.855-9.134-32.99-16.288-51.064-21.155
	c-1.664-21.533-19.651-38.49-41.61-38.49h-28.693c-21.959,0-39.945,16.957-41.61,38.49c-18.074,4.866-35.21,12.021-51.064,21.155
	c-16.402-14.05-41.113-13.324-56.641,2.204l-20.29,20.289c-15.528,15.528-16.255,40.239-2.204,56.641
	c-9.134,15.855-16.288,32.99-21.155,51.064c-21.533,1.665-38.49,19.651-38.49,41.611v28.693c0,21.959,16.957,39.945,38.49,41.61
	c4.867,18.074,12.021,35.21,21.155,51.064c-14.05,16.402-13.324,41.113,2.204,56.641l20.289,20.289
	c15.528,15.528,40.239,16.255,56.641,2.204c15.855,9.134,32.99,16.288,51.064,21.155c1.665,21.532,19.651,38.49,41.611,38.49h28.693
	c21.959,0,39.945-16.957,41.61-38.49c18.074-4.866,35.21-12.021,51.064-21.155c16.402,14.05,41.113,13.324,56.641-2.204
	l20.289-20.289c15.528-15.528,16.255-40.239,2.204-56.641c9.134-15.855,16.288-32.99,21.155-51.064
	c21.533-1.664,38.49-19.651,38.49-41.611v-28.693C501.801,219.694,484.844,201.708,463.311,200.043z"/>
<circle style="fill:#FFAD61;" cx="256" cy="256" r="170.837"/>
<path style="fill:#D35B38;" d="M267.255,162.242c0,0-2.558,4.946,7.674,10.232c4.166,2.152,11.809,4.248,19.407,5.977l0.731-11.207
	c1.007-15.449-11.253-28.534-26.735-28.534h-24.663c-15.482,0-27.743,13.086-26.735,28.535l1.096,16.807
	C256.888,181.891,267.255,162.242,267.255,162.242z"/>
<path style="fill:#FFE6B8;" d="M274.929,172.474c-10.232-5.286-7.674-10.232-7.674-10.232s-10.366,19.649-49.225,21.81l2.56,39.249
	c0.788,12.091,7.692,22.944,18.31,28.784l0,0c10.649,5.856,23.554,5.856,34.203,0l0,0c10.617-5.839,17.521-16.693,18.31-28.784
	l2.925-44.85C286.737,176.723,279.095,174.627,274.929,172.474z"/>
<path style="fill:#BCC987;" d="M353.722,332.091c-4.382-24.183-23.257-42.628-46.689-47.04c-3.526-0.664-7.154-1.013-10.852-1.013
	H283.88L256,311.919l-27.881-27.881h-12.301c-3.698,0-7.326,0.349-10.852,1.013c-23.431,4.412-42.307,22.856-46.688,47.04l0,0
	c-2.128,11.742,6.896,22.546,18.829,22.546h157.786C346.827,354.639,355.849,343.835,353.722,332.091L353.722,332.091z"/>
<g>
	<path style="fill:#4D3D36;" d="M471.323,190.954c-4.306-14.277-10.028-28.09-17.072-41.213
		c12.809-20.164,10.23-47.299-7.086-64.616l-20.289-20.289c-17.317-17.317-44.451-19.895-64.615-7.085
		c-13.122-7.044-26.935-12.766-41.212-17.071C315.846,17.362,294.836,0,270.346,0h-28.693c-24.488,0-45.5,17.362-50.7,40.679
		c-14.278,4.306-28.09,10.028-41.212,17.071c-20.165-12.81-47.3-10.231-64.616,7.085L64.835,85.125
		c-17.315,17.317-19.894,44.452-7.085,64.616c-7.044,13.121-12.766,26.934-17.072,41.212C17.362,196.154,0,217.165,0,241.654v28.693
		c0,24.489,17.362,45.499,40.677,50.7c4.306,14.278,10.029,28.091,17.072,41.212c-12.808,20.165-10.23,47.3,7.086,64.616
		l20.289,20.289c17.316,17.316,44.451,19.895,64.616,7.085c13.122,7.044,26.935,12.766,41.212,17.071
		c5.201,23.315,26.212,40.679,50.7,40.679h28.693c24.488,0,45.5-17.362,50.7-40.679c14.278-4.306,28.091-10.028,41.212-17.071
		c20.164,12.81,47.299,10.231,64.615-7.085l20.29-20.289c17.316-17.317,19.895-44.452,7.085-64.616
		c7.045-13.123,12.766-26.936,17.072-41.213c23.316-5.202,40.678-26.211,40.678-50.7v-28.693
		C512,217.165,494.639,196.154,471.323,190.954z M491.602,270.346c0,16.371-12.771,30.18-29.076,31.44
		c-4.308,0.333-7.939,3.345-9.063,7.518c-4.583,17.019-11.36,33.38-20.143,48.626c-2.158,3.745-1.721,8.444,1.091,11.727
		c10.64,12.421,9.905,31.217-1.67,42.794l-20.289,20.289c-11.577,11.577-30.373,12.309-42.795,1.671
		c-3.282-2.812-7.981-3.249-11.727-1.091c-15.245,8.784-31.604,15.561-48.625,20.143c-4.172,1.124-7.183,4.754-7.517,9.063
		c-1.261,16.304-15.071,29.076-31.442,29.076h-28.693c-16.37,0-30.18-12.771-31.441-29.077c-0.333-4.309-3.344-7.939-7.517-9.063
		c-17.02-4.583-33.38-11.36-48.625-20.143c-1.586-0.914-3.341-1.362-5.089-1.362c-2.379,0-4.745,0.831-6.637,2.453
		c-12.422,10.64-31.219,9.905-42.795-1.671L79.26,412.45c-11.576-11.577-12.309-30.374-1.671-42.795
		c2.812-3.283,3.249-7.982,1.091-11.727c-8.783-15.244-15.56-31.603-20.143-48.626c-1.124-4.173-4.755-7.183-9.063-7.517
		c-16.304-1.259-29.076-15.068-29.076-31.439v-28.693c0-16.371,12.771-30.18,29.076-31.44c4.308-0.334,7.939-3.344,9.063-7.517
		c4.584-17.021,11.361-33.381,20.143-48.626c2.158-3.745,1.721-8.444-1.091-11.727c-10.64-12.421-9.905-31.218,1.67-42.795
		l20.289-20.289c11.577-11.578,30.374-12.309,42.795-1.671c3.283,2.813,7.982,3.249,11.727,1.091
		c15.245-8.784,31.604-15.561,48.625-20.143c4.173-1.124,7.183-4.754,7.517-9.063c1.261-16.303,15.071-29.075,31.442-29.075h28.693
		c16.37,0,30.181,12.771,31.442,29.077c0.333,4.309,3.344,7.939,7.517,9.063c17.019,4.583,33.379,11.36,48.625,20.143
		c3.745,2.158,8.444,1.722,11.727-1.091c12.421-10.641,31.218-9.906,42.794,1.671l20.289,20.289
		c11.576,11.577,12.31,30.373,1.671,42.794c-2.812,3.283-3.249,7.982-1.091,11.727c8.784,15.246,15.561,31.606,20.143,48.626
		c1.124,4.173,4.755,7.184,9.063,7.518c16.303,1.259,29.075,15.068,29.075,31.439V270.346z"/>
	<path style="fill:#4D3D36;" d="M256,74.964c-99.824,0-181.036,81.212-181.036,181.036S156.176,437.036,256,437.036
		S437.036,355.824,437.036,256S355.824,74.964,256,74.964z M256,416.637c-88.576,0-160.637-72.061-160.637-160.637
		S167.424,95.363,256,95.363S416.637,167.424,416.637,256S344.576,416.637,256,416.637z"/>
	<path style="fill:#4D3D36;" d="M207.852,184.715l2.56,39.249c1.02,15.648,9.832,29.5,23.571,37.057
		c6.893,3.792,14.454,5.687,22.016,5.687c7.562,0,15.123-1.895,22.017-5.687c13.739-7.557,22.55-21.409,23.571-37.057l2.925-44.851
		l0.73-11.206c0.663-10.167-2.953-20.262-9.919-27.698c-6.966-7.435-16.805-11.7-26.993-11.7h-24.662
		c-10.189,0-20.028,4.265-26.994,11.701c-6.967,7.435-10.582,17.53-9.919,27.697L207.852,184.715z M268.187,243.147
		c-7.631,4.197-16.742,4.197-24.372,0.001c-7.606-4.184-12.483-11.85-13.047-20.511l-1.918-29.42
		c17.919-2.55,29.589-8.691,36.863-14.424c1.361,0.967,2.873,1.882,4.535,2.741c3.077,1.59,7.485,3.163,13.356,4.762l-2.37,36.34
		C280.669,231.298,275.791,238.965,268.187,243.147z M284.925,165.435c-3.081-0.973-4.625-1.667-5.314-2.021
		c-0.643-0.331-1.151-0.631-1.566-0.895l2.91-7.759C283.523,157.767,284.918,161.492,284.925,165.435z M231.561,154.157
		c3.171-3.384,7.471-5.249,12.108-5.249h17.694l-3.303,8.807c-1.53,2.201-9.235,11.666-30.542,15.09l-0.406-6.226
		C226.81,161.953,228.39,157.541,231.561,154.157z"/>
	<path style="fill:#4D3D36;" d="M308.92,275.029c-4.193-0.788-8.479-1.189-12.738-1.189h-12.301c-2.705,0-5.299,1.075-7.212,2.987
		L256,297.495l-20.668-20.669c-1.912-1.912-4.507-2.987-7.212-2.987h-12.301c-4.259,0-8.545,0.401-12.74,1.189
		c-27.76,5.227-49.798,27.429-54.837,55.245c-1.554,8.58,0.76,17.34,6.349,24.032c5.589,6.694,13.795,10.532,22.516,10.532h157.786
		c8.719,0,16.927-3.839,22.516-10.531c5.589-6.694,7.903-15.453,6.349-24.032C358.718,302.457,336.681,280.256,308.92,275.029z
		 M341.752,341.231c-1.001,1.198-3.222,3.209-6.859,3.209H177.107c-3.636,0-5.858-2.01-6.859-3.209s-2.582-3.743-1.934-7.322
		c3.542-19.554,19.031-35.161,38.539-38.834c2.953-0.555,5.97-0.836,8.966-0.836h8.077l24.892,24.893
		c1.912,1.912,4.507,2.987,7.212,2.987c2.705,0,5.299-1.075,7.212-2.987l24.893-24.893h8.077c2.997,0,6.013,0.281,8.964,0.836
		c19.509,3.675,34.998,19.281,38.54,38.834C344.334,337.489,342.752,340.033,341.752,341.231z"/>
	<path style="fill:#4D3D36;" d="M363.17,173.038c-4.698,3.108-5.988,9.435-2.88,14.133c13.51,20.426,20.65,44.227,20.65,68.829
		c0,5.632,4.566,10.199,10.199,10.199s10.199-4.567,10.199-10.199c0-28.617-8.311-56.309-24.034-80.082
		C374.196,171.221,367.868,169.93,363.17,173.038z"/>
	<path style="fill:#4D3D36;" d="M336.871,160.759c1.43,1.216,2.857,2.487,4.242,3.775c1.965,1.83,4.459,2.734,6.947,2.734
		c2.733,0,5.46-1.091,7.468-3.25c3.838-4.124,3.606-10.577-0.516-14.415c-1.608-1.497-3.266-2.972-4.927-4.385
		c-4.292-3.648-10.728-3.128-14.377,1.164C332.059,150.674,332.579,157.111,336.871,160.759z"/>
</g>
</svg>
''',
    "user_activity_event": ''' 
    <svg height="800px" width="800px" version="1.1" id="Layer_1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" 
	 viewBox="0 0 512 512" xml:space="preserve">
<path style="fill:#D35B38;" d="M283.174,69.982c0,0-6.176,11.94,18.528,24.703c10.058,5.196,28.511,10.256,46.855,14.43
	l1.764-27.056c2.433-37.303-27.168-68.895-64.549-68.895h-59.546c-37.381,0-66.981,31.593-64.549,68.895l2.647,40.581
	C258.144,117.421,283.174,69.982,283.174,69.982z"/>
<path style="fill:#FFE6B8;" d="M301.702,94.686c-24.704-12.763-18.528-24.703-18.528-24.703s-25.03,47.44-118.849,52.657
	l6.18,94.763c1.904,29.193,18.572,55.397,44.206,69.495l0,0c25.71,14.14,56.869,14.14,82.579,0l0,0
	c25.634-14.098,42.302-40.302,44.206-69.495l7.062-108.287C330.213,104.942,311.76,99.882,301.702,94.686z"/>
<path style="fill:#BCC987;" d="M491.939,444.401c-10.578-58.388-56.153-102.92-112.725-113.572L256,454.044L132.786,330.829
	C76.214,341.481,30.638,386.014,20.061,444.401l-9.862,54.436h491.602L491.939,444.401z"/>
<g>
	<path style="fill:#FFE6B8;" d="M379.215,330.829c-8.512-1.603-17.271-2.446-26.201-2.446h-29.699L256,395.697l-67.314-67.314h-29.7
		c-8.929,0-17.688,0.843-26.201,2.446L256,454.044L379.215,330.829z"/>
</g>
<g>
	<path style="fill:#4D3D36;" d="M154.148,123.303l6.181,94.763c2.141,32.837,20.633,61.91,49.468,77.769
		c14.469,7.957,30.333,11.934,46.204,11.934c15.868,0,31.74-3.979,46.204-11.933c28.834-15.86,47.327-44.933,49.468-77.77
		l7.062-108.285v-0.001l1.764-27.058c1.363-20.886-5.769-40.799-20.079-56.072c-14.309-15.274-33.718-23.686-54.647-23.686h-59.546
		c-20.93,0-40.338,8.411-54.647,23.686c-14.309,15.273-21.441,35.186-20.079,56.072L154.148,123.303z M292.373,277.962
		c-22.775,12.526-49.972,12.527-72.747-0.001c-22.7-12.484-37.258-35.372-38.945-61.222l-5.523-84.681
		c57.446-5.269,88.778-26.267,104.215-41.341c4.128,4.734,10.025,9.094,17.645,13.031c8.232,4.253,21.934,8.723,40.798,13.318
		l-6.5,99.673C329.631,242.59,315.074,265.477,292.373,277.962z M325.535,40.596c10.411,11.113,15.601,25.603,14.609,40.798
		l-0.978,14.994c-18.902-4.751-28.216-8.403-32.783-10.763c-11.121-5.747-13.632-10.5-13.699-11.959
		c0.008-0.02,0.013-0.041,0.02-0.061c0.006-0.015,0.013-0.029,0.018-0.044l16.797-44.793
		C315.447,31.643,320.866,35.613,325.535,40.596z M186.465,40.596c10.412-11.113,24.533-17.233,39.762-17.233h59.546
		c1.321,0,2.629,0.063,3.932,0.154l-15.776,42.071c-1.313,2.119-7.634,11.502-22.063,21.244
		c-14.09,9.513-38.767,21.163-78.034,24.871l-1.977-30.308C170.864,66.199,176.054,51.71,186.465,40.596z"/>
	<path style="fill:#4D3D36;" d="M511.837,497.02l-9.862-54.435c-5.527-30.507-20.03-58.2-41.943-80.086
		c-21.655-21.629-48.947-36.045-78.924-41.69c-0.001,0-0.003-0.001-0.004-0.001c-9.238-1.74-18.689-2.622-28.09-2.622h-29.699
		c-2.705,0-5.298,1.075-7.212,2.987L256,381.273l-60.102-60.103c-1.913-1.912-4.507-2.987-7.212-2.987h-29.7
		c-9.401,0-18.851,0.882-28.09,2.622c-0.001,0-0.003,0.001-0.004,0.001c-29.977,5.645-57.27,20.061-78.926,41.69
		c-21.912,21.886-36.415,49.579-41.942,80.086L0.163,497.02c-0.539,2.974,0.27,6.035,2.207,8.355
		c1.938,2.319,4.806,3.661,7.829,3.661h491.602c3.023,0,5.89-1.341,7.829-3.661C511.567,503.055,512.375,499.994,511.837,497.02z
		 M327.539,338.582h25.476c1.321,0,2.642,0.02,3.963,0.061l-50.488,50.488L256,439.619l-14.749-14.749l21.961-21.961
		L327.539,338.582z M184.461,338.582l57.115,57.115l-14.749,14.749l-21.315-21.315l-50.488-50.488
		c1.322-0.041,2.643-0.061,3.963-0.061L184.461,338.582L184.461,338.582z M22.413,488.638l7.684-42.418
		c9.295-51.308,48.902-92.598,99.378-104.277l119.313,119.312c1.992,1.991,4.602,2.987,7.212,2.987c2.61,0,5.22-0.996,7.212-2.987
		l119.313-119.312c50.475,11.68,90.082,52.971,99.378,104.277l7.684,42.418H22.413z"/>
</g>
</svg>
'''
}

# Encode the icons.
encoded_icons = {key: encode_svg(svg) for key, svg in svg_icons.items()}

from collections import defaultdict
import pandas as pd
from pyvis.network import Network
from IPython.display import HTML, display
from pyspark.sql.functions import col

def visualize_interactive_graph(
    nodes_df,
    edges_df,
    tenant_settings_df,
    selected_settings=["all"],
    sample_n=200
):
    """
    Visualizes only the subgraph for the given list of tenant-setting names.
    Pass selected_settings=["all"] to show all settings,
    ["alldelegable"] to show only delegable settings (at any level),
    or a list of specific setting names to filter.
    """
    # 1) Load & clean
    nodes_pd = nodes_df.limit(1000).toPandas().replace("", pd.NA)
    edges_pd = edges_df.limit(1000).toPandas().replace("", pd.NA)
    nodes_pd.dropna(subset=["node_id", "node_type"], inplace=True)
    edges_pd.dropna(subset=["src_node", "dest_node"], inplace=True)

    # 2) Fetch delegability flags and base tenant-setting IDs
    ts_pd = tenant_settings_df.select(
        "node_id", "delegateToWorkspace", "delegateToCapacity", "delegateToDomain"
    ).toPandas()
    ts_ids = set(ts_pd["node_id"])
    all_ts_nodes = set(nodes_pd[nodes_pd.node_type == "tenant_setting"]["node_id"])

    # 3) Determine allowed_settings
    if selected_settings == ["all"]:
        allowed_settings = all_ts_nodes
    elif selected_settings == ["alldelegable"]:
        mask = ts_pd[["delegateToWorkspace","delegateToCapacity","delegateToDomain"]]\
            .fillna(False).any(axis=1)
        base_allowed = set(ts_pd.loc[mask, "node_id"])
        allowed_settings = {nid for nid in all_ts_nodes if any(nid.startswith(base) for base in base_allowed)}
    else:
        prefixes = {"tenant_setting_" + s for s in selected_settings}
        allowed_settings = {nid for nid in all_ts_nodes if any(nid.startswith(p) for p in prefixes)}

    # 4) Start with full node set
    nodes_sub = nodes_pd.copy()

    # 5) Prune only unwanted tenant_setting & orphan activities
    if allowed_settings != all_ts_nodes:
        mask_ts = nodes_sub["node_type"] == "tenant_setting"
        nodes_sub = nodes_sub.loc[(~mask_ts) | (nodes_sub["node_id"].isin(allowed_settings))].copy()

        mask_act = nodes_sub["node_type"] == "activity"
        keep_acts = set(
            edges_pd.loc[
                (edges_pd.relationship=="modified_by") &
                (edges_pd.src_node.isin(allowed_settings)),
                "dest_node"
            ]
        )
        nodes_sub = nodes_sub.loc[(~mask_act) | (nodes_sub["node_id"].isin(keep_acts))].copy()

    # 6) Rebuild edges_sub
    valid_ids = set(nodes_sub["node_id"])
    edges_sub = edges_pd.loc[
        edges_pd["src_node"].isin(valid_ids) &
        edges_pd["dest_node"].isin(valid_ids)
    ].copy()

    # 7) Sample
    if sample_n < len(nodes_sub):
        nodes_sub = nodes_sub.sample(n=sample_n, random_state=42)
    if sample_n < len(edges_sub):
        edges_sub = edges_sub.sample(n=sample_n, random_state=42)

    # 8) Aggregated counts for halo
    act_edges = edges_sub[edges_sub.relationship=="modified_by"]
    raw_counts = act_edges.groupby("src_node").size().to_dict()
    agg_counts = defaultdict(int)
    for nid, c in raw_counts.items():
        base = nid
        for suf in ("_domain_","_capacity_","_workspace_"):
            if suf in base:
                base = base.split(suf)[0]
        agg_counts[base] += c

    # 9) Render with PyVis
    net = Network(height="1080px", width="100%", directed=True)
    net.set_options("""
        var options = {
            "nodes": {"size":40,"font":{"size":20}},
            "layout":{"hierarchical":{"enabled":false}},
            "physics":{
                "barnesHut":{"gravitationalConstant":-8000,"centralGravity":0.3,
                             "springLength":250,"springConstant":0.001,
                             "damping":0.09,"avoidOverlap":0.7},
                "minVelocity":0.75
            }
        }
    """)

    for _, row in nodes_sub.iterrows():
        nt, nid = row.node_type, row.node_id

        if nt == "tenant_setting":
            simple = nid.replace("tenant_setting_","").split("_")[0]
            base = next((nid.split(s)[0] for s in ("_domain_","_capacity_","_workspace_") if s in nid), nid)
            total = agg_counts.get(base,0)
            kwargs = {"label": simple, "title": simple,
                      "shape":"image", "image":encoded_icons["tenant_settings"]}
            if total > 2:
                kwargs.update(borderWidth=6, shadow={
                    "enabled":True, "color":"rgba(255,0,0,0.5)",
                    "size":10, "x":0, "y":0
                })
            net.add_node(nid, **kwargs)

        elif nt == "tenant":
            net.add_node(nid, label=nid, title=nt, shape="image", image=encoded_icons["tenant"])

        elif nt == "capacity":
            capacity_id = nid.replace("capacity_","")
            cap_records = capacities_df.filter(col("id")==capacity_id).collect()
            if cap_records:
                cap = cap_records[0]
                label = cap["displayName"]
                tooltip = (f"Capacity ID: {cap['id']}\n"
                           f"Display Name: {cap['displayName']}\n"
                           f"Type: {cap['sku']}")
            else:
                label, tooltip = capacity_id, "No capacity record found for the given ID."
            net.add_node(nid, label=label, title=tooltip,
                         shape="image", image=encoded_icons["capacity"])

        elif nt == "domain":
            domain_id = nid.replace("domain_","")
            dom_records = domains_df.filter(col("id")==domain_id).collect()
            if dom_records:
                dom = dom_records[0]
                label = dom["displayName"]
                tooltip = (f"Domain ID: {dom['id']}\n"
                           f"Display Name: {dom['displayName']}\n"
                           f"Parent: {dom['parentDomainId']}")
            else:
                label, tooltip = domain_id, "No domain record found for the given ID."
            net.add_node(nid, label=label, title=tooltip,
                         shape="image", image=encoded_icons["domain"])

        elif nt == "workspace":
            ws_id = nid.replace("workspace_","")
            ws_records = workspaces_df.filter(col("id")==ws_id).collect()
            if ws_records:
                ws = ws_records[0]
                label = ws["name"]
                tooltip = (f"Workspace ID: {ws['id']}\n"
                           f"Workspace Name: {ws['name']}\n"
                           f"Type: {ws['type']}")
            else:
                label, tooltip = ws_id, "No workspace record found for the given ID."
            net.add_node(nid, label=label, title=tooltip,
                         shape="image", image=encoded_icons["workspace"])

        elif nt == "activity":
            act_id = nid.replace("activity_","")
            act_records = activity_df.filter(col("Id")==act_id).collect()
            if act_records:
                act = act_records[0]
                label = act["UserId"]
                tooltip = (f"Activity ID: {act['Id']}\n"
                           f"Creation Time: {act['CreationTime']}\n"
                           f"Client IP: {act['ClientIP']}\n"
                           f"Details: {act['TenantAdminFeatureSwitches']}")
            else:
                label, tooltip = act_id, "No activity record found for the given ID."
            net.add_node(nid, label=label, title=tooltip,
                         shape="image", image=encoded_icons["user_activity_event"])

        else:
            net.add_node(nid, label=nid, title=nt)

    for _, e in edges_sub.iterrows():
        net.add_edge(e.src_node, e.dest_node, title=e.relationship)

    net.write_html("interactive_graph.html")
    display(HTML(open("interactive_graph.html").read()))

# Example usage:
visualize_interactive_graph(nodes_all, edges_all, tenant_settings_df,
                            selected_settings=["alldelegable"], sample_n=200)


# Example usage
'''
visualize_interactive_graph(
    nodes_all,
    edges_all,
    tenant_settings_df,
    selected_settings=["ExportToImage","ExportToXML"],
    sample_n=1000
)
'''
    
'''
visualize_interactive_graph(
    nodes_all,
    edges_all,
    tenant_settings_df,
    selected_settings=["alldelegable"],
    sample_n=1000
)
'''

'''
visualize_interactive_graph(
    nodes_all,
    edges_all,
    tenant_settings_df,
    selected_settings=["all"],
    sample_n=1000
)
'''





StatementMeta(, 5eaf4197-12de-4933-ad40-7944a0b939b2, 25, Finished, Available, Finished)

'\nvisualize_interactive_graph(\n    nodes_all,\n    edges_all,\n    tenant_settings_df,\n    selected_settings=["all"],\n    sample_n=1000\n)\n'